In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Direktori dataset
dataset_dir = 'dataset/'

# Parameter
img_width, img_height = 128, 128
batch_size = 32

# ImageDataGenerator untuk augmentasi dan normalisasi
datagen = ImageDataGenerator(
    rescale=1.0 / 255,        # Normalisasi
    validation_split=0.2      # 80% training, 20% validasi
)

# Training dan validasi generator
train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True  # Data akan diacak
)

val_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

# Mendapatkan label (mapping folder ke indeks)
label_map = train_generator.class_indices
# Membalik dictionary agar indeks ke label
label_map_reverse = {v: k for k, v in label_map.items()}

print("Label Mapping:", label_map)


Found 386 images belonging to 4 classes.
Found 94 images belonging to 4 classes.
Label Mapping: {'anger': 0, 'happy': 1, 'neutral': 2, 'sadness': 3}


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Membuat model CNN
model = Sequential([
    # Layer konvolusi pertama
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    # Layer konvolusi kedua
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # Layer konvolusi ketiga
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # Flatten dan Fully Connected Layer
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Dropout untuk mencegah overfitting
    Dense(len(label_map), activation='softmax')  # Output sesuai jumlah kelas
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,305,156 (12.61 MB)

 Trainable params: 3,305,156 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# Melatih model
history = model.fit(
    train_generator,
    epochs=10,  # Sesuaikan jumlah epoch
    validation_data=val_generator
)


Epoch 1/10


c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 265ms/step - accuracy: 0.3098 - loss: 1.9702 - val_accuracy: 0.4362 - val_loss: 1.2333
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 260ms/step - accuracy: 0.4272 - loss: 1.2214 - val_accuracy: 0.6383 - val_loss: 1.0715
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 268ms/step - accuracy: 0.6210 - loss: 1.0251 - val_accuracy: 0.6596 - val_loss: 0.7070
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 256ms/step - accuracy: 0.7872 - loss: 0.5526 - val_accuracy: 0.8085 - val_loss: 0.5461
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 255ms/step - accuracy: 0.8048 - loss: 0.5199 - val_accuracy: 0.8191 - val_loss: 0.3563
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 257ms/step - accuracy: 0.8712 - loss: 0.3196 - val_accuracy: 0.8830 - val_loss: 0.3478
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 278ms/step - accuracy: 0.9069 - loss: 0.2652 - val_accuracy: 0.8617 - val_loss: 0.3516
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 252ms/step - accuracy: 0.9540 - loss: 0.1445 - val_accuracy: 0.8298 - val_

In [7]:
import pickle
# Simpan model dalam format .keras
model.save('emotion_detection.keras')

# Simpan label mapping
with open('label_map.pkl', 'wb') as f:
    pickle.dump(label_map, f)

print("Model dan label mapping berhasil disimpan!")


Model dan label mapping berhasil disimpan!


In [8]:
from tensorflow.keras.models import load_model
# Memuat model yang sudah disimpan
loaded_model = load_model('emotion_detection.keras')

# Memuat label mapping
with open('label_map.pkl', 'rb') as f:
    loaded_label_map = pickle.load(f)
    loaded_label_map_reverse = {v: k for k, v in loaded_label_map.items()}

print("Model dan label mapping berhasil dimuat!")


Model dan label mapping berhasil dimuat!


In [9]:
from tensorflow.keras.preprocessing import image

def predict_emotion(img_path):
    # Memuat dan preprocess gambar
    img = image.load_img(img_path, target_size=(img_width, img_height))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Prediksi menggunakan model yang dimuat
    predictions = loaded_model.predict(img_array)
    class_idx = np.argmax(predictions)
    class_label = loaded_label_map_reverse[class_idx]

    return class_label, predictions[0][class_idx]


In [16]:
# Path gambar untuk testing
test_img_path = 'anger2_test.png'

# Prediksi emosi
predicted_class, confidence = predict_emotion(test_img_path)
print(f"Predicted Emotion: {predicted_class} with confidence {confidence}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Predicted Emotion: anger with confidence 0.9870566129684448
